In [50]:
import requests
import json
from bs4 import BeautifulSoup
from typing import List
import time
from concurrent.futures import ThreadPoolExecutor
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def extract_urls_lists(search_page_url: str) -> List[str]:
    response = requests.get(search_page_url, params = {})
    soup = BeautifulSoup(response.text, 'html.parser')

    list_of_urls=[]

    for article in soup.find_all('iw-search-card-rendered'):
        for link in article.find_all('a'):
            list_of_urls.append(link.get('href'))
    return list_of_urls

def extract_x_urls(number_of_urls: int=30) -> List[str]:
    immo_base_url = "https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page="
    immo_end_url = "&orderBy=relevance"
    searchpages_urls_list =[]

    for index in range(1,round(min(334,number_of_urls/30 + 1))):
        full_url = immo_base_url + str(index) + immo_end_url
        searchpages_urls_list.append(full_url)
    
    full_urls_list =[]

    with ThreadPoolExecutor() as pool:
        full_urls_list=list(pool.map(extract_urls_lists, searchpages_urls_list))    
    return sum(full_urls_list,[])
extract_x_urls(30)


['https://www.immoweb.be/en/classified/apartment/for-sale/deinze/9800/10562026',
 'https://www.immoweb.be/en/classified/apartment/for-sale/deinze/9800/10560242',
 'https://www.immoweb.be/en/classified/penthouse/for-sale/bruxelles/1130/10560964',
 'https://www.immoweb.be/en/classified/apartment/for-sale/evere/1140/10560966',
 'https://www.immoweb.be/en/classified/apartment/for-sale/ninove/9400/10563055',
 'https://www.immoweb.be/en/classified/apartment/for-sale/etterbeek/1040/10550578',
 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/deinze/9800/10563685',
 'https://www.immoweb.be/en/classified/apartment-block/for-sale/henri-chapelle/4841/10130670',
 'https://www.immoweb.be/en/classified/house/for-sale/henri-chapelle/4841/10130669',
 'https://www.immoweb.be/en/classified/flat-studio/for-sale/woluwe-saint-pierre/1150/10563238',
 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/evergem%20sleidinge/9940/10563592',
 'https:

In [48]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
pd.set_option('display.width', None)
def make_one_data_frame(url):
  """
  session = requests.Session()
  retry = Retry(connect=3, backoff_factor=0.5)
  adapter = HTTPAdapter(max_retries=retry)
  session.mount('http://', adapter)
  session.mount('https://', adapter)
  """
  response = requests.get(url)
  test_page = response.text

  dfs = pd.read_html(test_page)

  full_df = pd.concat(dfs).dropna(thresh=2).T
  full_df.columns = full_df.iloc[0]
  full_df = full_df[1:]
  full_df = full_df.loc[:, full_df.columns.isin(['Neighbourhood or locality','Building condition','Number of frontages','Living area','Kitchen type','Bedrooms','Bathrooms','Furnished','Surface of the plot','Garden surface','Swimming pool','Price','Terrace'])]

  return full_df
make_one_data_frame("https://www.immoweb.be/en/classified/house/for-sale/kortrijk/8500/10564478")

#data.to_csv("datanew.csv")
  # pd.read_html reads in all tables and returns a list of DataFrames



,Number of frontages,Living area,Bedrooms,Bathrooms,Furnished,Price
1,2,112 m² square meters,2,1,No,"€ 190,000 190000 €"


In [58]:

list_of_urls = extract_x_urls(3000)
print(len(list_of_urls))
print('list ready')


dataframes_list = []
with ThreadPoolExecutor() as pool:
    results = list(pool.map(make_one_data_frame, list_of_urls))
    for result in results:
        dataframes_list.append(result)

full_df = pd.concat(dataframes_list).reset_index(drop=True)
full_df.dropna(thresh=2,inplace=True)


    

print('done extracting')

#full_df.to_csv("df.csv")
full_df

#full_df.loc[:, full_df.columns.isin(['Neighbourhood or locality','Building condition','Number of frontages','Living area','Kitchen type','Bedrooms','Bathrooms','Furnished','Surface of the plot','Garden surface','Swimming pool','Price','Terrace'])]


3000
list ready
done extracting


,Neighbourhood or locality,Building condition,Number of frontages,Living area,Kitchen type,Bedrooms,Bathrooms,Furnished,Swimming pool,Price,Terrace,Surface of the plot,Garden surface
0,Gent en omstreken,As new,2,99 m² square meters,Hyper equipped,2,1,No,No,"€ 365,000 365000 €",NaN,NaN,NaN
1,1 - Bruxelles Centre,As new,3,148 m² square meters,USA hyper equipped,3,2,No,No,"€ 450,000 450000 €",NaN,NaN,NaN
2,1 - Bruxelles Est,As new,2,140 m² square meters,USA hyper equipped,3,2,No,No,"€ 595,000 595000 €",NaN,NaN,NaN
3,NaN,Good,2,60 m² square meters,Semi equipped,1,1,No,NaN,"€ 150,000 150000 €",Yes,NaN,NaN
4,1 - Bruxelles Est,Good,NaN,41 m² square meters,Installed,1,1,No,No,"€ 195,000 195000 €",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Bruxelles NORD,As new,3,189 m² square meters,Hyper equipped,4,1,No,No,"€ 530,000 530000 €",NaN,306 m² square meters,60 m² square meters
2996,Zone Centre Province du Luxembourg,Good,3,168 m² square meters,USA hyper equipped,4,1,No,Yes,"€ 530,000 530000 €",Yes,816 m² square meters,384 m² square meters
2997,NaN,As new,4,219 m² square meters,Hyper equipped,4,1,No,NaN,"€ 540,000 540000 €",NaN,786 m² square meters,600 m² square meters
2998,NaN,Just renovated,2,213 m² square meters,Semi equipped,4,2,No,NaN,"€ 540,000 540000 €",NaN,545 m² square meters,360 m² square meters
